<div>
    <img src="images/emlyon.png" style="height:60px; float:left; padding-right:10px; margin-top:5px" />
    <span>
        <h1 style="padding-bottom:5px;"> Introduction to Deep Learning </h1>
        <a href="https://masters.em-lyon.com/fr/msc-in-data-science-artificial-intelligence-strategy">[DSAIS]</a> MSc in Data Science & Artificial Intelligence Strategy <br/>
         Paris | © Saeed VARASTEH
    </span>
</div>

<div class="alert-info" style="border-bottom: solid 1px lightgray; background-color:#ece4f5;">
    <img src="images/assignment.png" style="height:60px; float:left; padding-right:10px;" />
    <span style="font-weight:bold; color:#8966b0">
        <h4 style="padding-top:25px;"> FINAL PROJECT </h4>
    </span>
</div>

###  DSAIS Face Recognition Application


In this project, you'll build a facial recognition system to recognize all of your friends.

Face recognition problems generally fall into one of two categories:

- __Face verification__, where you build a model to distinguish your face from others. This is a binary classification problem.
- __Face Recognition__, where you build a model to identify multiple faces. This is a multi-class classification problem.

Imagine a system for the Emlyon building where we want to provide face recognition to allow students to enter the building.

The goal is to fine-tune the classification head of __FaceNet__ to classify all your friends.

The project consists of two phases. 

- __Face Detection:__ In the first phase, you will need to prepare your datasets; you will use the __MTCNN__ model to create cropped face images from your raw images and store them in new directories.

- __Face Recognition:__ The second phase is very similar to what we did with transfer learning. You modify the classifier head of __FaceNet__ and train it on the cropped face images.

You will find FaceNet documentation here: [FaceNet](https://github.com/timesler/facenet-pytorch)

Finally, you can use your trained model to test it on test data.

__Extra step (high bounce!)__:

Build a Python application that shows recognized faces of DSAIS students on video feeds (or real-time images from the camera). 

---

### Getting face samples

For this project, you will need eleven photos of yourself and each of your friends on which to base your facial recognition application. 

You will have to work together to get them. 

__The sooner you take your photos and share them with others, the easier it will be for everyone to start working on the project.__

Let us say the deadline for uploading your pictures is <span style="color:crimson">Sunday, February 25, 23:59.</span>

Once you have your photos, post them [here](https://drive.google.com/drive/folders/1O8InPDGGy1eK7qjT92rL0DU9nd16lbo8?usp=sharing) under your name:

#### Names

In [2]:
Names = ['Gautier'  'Alexandre'  'Chadi'  'Ruicong'  'Shiqing'  'Namrata'  'Martino'  'Romain'
         'Fulin'  'Eze'  'Tristan'  'Braxton'  'Gabriel'  'JeanLuis'
         'Benjamin'  'Xinyue'  'Qinyi'  'JiaXin'  'Qiqi'  'Charlie'  'AswinSri'
         'KingMan'  'Marouane'  'SongRim'  'Selin'  'HoangMinhThu'  'Selle'  'Peiwen'
         'JiyuanPeter'  'Jing'  'Muyun'  'Mathieu'  'Michelle'  'Shiyao'  'Nicholas'
         'Thomas'  'Maria'  'Jad'  'Ata'  'Manuel'  'Yuekai'  'Wujiachen'
         'Saeed'  'Ahmed'  'Valentin'  'Mohamed']

#### Set Name

In [6]:
import os
your_name = "Fulin" # replace it with one of the names above (your name)
try:  
    os.mkdir(your_name) 
    print("Directory created.")
except OSError as error:  
    print("The directory exists.")   

Directory created.


#### OpenCV Library

Install the opencv library if you do not have it. It is needed to access the camera on your machine.

In [1]:
# !pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/77/df/b56175c3fb5bc058774bdcf35f5a71cf9c3c5b909f98a1c688eb71cd3b1f/opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 82.9 MB/s eta 0:00:00:00:010:01m


#### Get Samples

The following code will allow you to access your system camera.

Once you have the feeds. Press "__s__" to save a new picture of your face.

You will need to do this 11 times.

When you are done, press "__q__" to close the camera window.

In [7]:
import cv2
import time

video_capture = cv2.VideoCapture(0)
src = your_name+"/"
    
while True:
    _, frame = video_capture.read()
    
    scale_percent = 50
    dim = (int(frame.shape[1] * scale_percent / 100), int(frame.shape[0] * scale_percent / 100))
    frame = cv2.resize(frame, dim)
    
    cv2.imshow('Video Stream', frame)
    
    key = cv2.waitKey(1) & 0xff

    if key  == ord('s'):
        timestr = time.strftime("%Y-%m-%d %H:%M:%S")
        print("Saving image: "+timestr + ".jpg")
        filename = src + timestr + ".jpg"
        cv2.imwrite(filename, frame)
        
    if key == ord('q'):
        video_capture.release()
        break

cv2.destroyAllWindows()
cv2.waitKey(1)
print("Done")

Saving image: 2024-03-05 09:11:37.jpg
Saving image: 2024-03-05 09:11:38.jpg
Saving image: 2024-03-05 09:11:39.jpg
Saving image: 2024-03-05 09:11:40.jpg
Saving image: 2024-03-05 09:11:41.jpg
Saving image: 2024-03-05 09:11:42.jpg
Saving image: 2024-03-05 09:11:43.jpg
Saving image: 2024-03-05 09:11:44.jpg
Saving image: 2024-03-05 09:11:48.jpg
Saving image: 2024-03-05 09:11:49.jpg
Saving image: 2024-03-05 09:11:50.jpg
Done


----

### Phase 01: Face detection

In [22]:
from PIL import Image
import pillow_heif
from facenet_pytorch import MTCNN
import torch
import os
import glob

# 初始化MTCNN
mtcnn = MTCNN(keep_all=True, post_process=False)

# 读取HEIC文件并转换为PIL图像
def read_heic_image(file_path):
    heif_file = pillow_heif.read_heif(file_path)  # 确保你使用的pillow_heif版本与这里的使用方法相匹配
    image = Image.frombytes(
        heif_file.mode, 
        heif_file.size, 
        heif_file.data,
        "raw",
    )
    return image

# 检测并裁剪图像中的面部，并保存为JPEG格式
def process_and_save_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 遍历所有文件
    for path in glob.glob(f'{input_dir}/**/*', recursive=True):
        if os.path.isdir(path):
            continue  # Skip directories
        print(f'Processing: {path}')
        try:
            # Load image, converting to RGB if necessary
            if path.lower().endswith('.heic'):
                image = read_heic_image(path).convert('RGB')
            else:
                image = Image.open(path).convert('RGB')  # Convert image to RGB
            
            # Use MTCNN to detect faces
            boxes, _ = mtcnn.detect(image)
            if boxes is not None:
                for i, box in enumerate(boxes):
                    face = image.crop(box)
                    # Save the cropped face image as JPEG
                    file_name = os.path.basename(path).split('.')[0] + f'_face{i}.jpg'
                    person_dir = os.path.join(output_dir, os.path.basename(os.path.dirname(path)))
                    if not os.path.exists(person_dir):
                        os.makedirs(person_dir)
                    face.save(os.path.join(person_dir, file_name), format='JPEG')
        except Exception as e:
            print(f"Error processing {path}: {e}")

# 设置数据目录
input_dir = './samples'
output_dir = './faces'

process_and_save_images(input_dir, output_dir)

Processing: ./samples/Gautier/2024-02-24 18-08-12.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-13.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-07.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-29.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-15.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-49.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-44.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-46.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-21.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-20.jpg
Processing: ./samples/Gautier/2024-02-24 18-08-23.jpg
Processing: ./samples/Alexandre/WIN_20240223_16_57_53_Pro.jpg
Processing: ./samples/Alexandre/WIN_20240223_16_57_53_Pro (2).jpg
Processing: ./samples/Alexandre/WIN_20240223_16_57_42_Pro.jpg
Processing: ./samples/Alexandre/WIN_20240223_16_57_52_Pro.jpg
Processing: ./samples/Alexandre/WIN_20240223_16_57_49_Pro.jpg
Processing: ./samples/Alexandre/WIN_20240223_16_57_50_Pro.jpg
Processing: ./samples/Alexandr

### Phase 02: Face recognition 

In [4]:
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split

def split_data(src_folder, dst_folder):
    # Create directories for train, val, and test inside the destination folder
    train_dir = os.path.join(dst_folder, 'train')
    val_dir = os.path.join(dst_folder, 'val')
    test_dir = os.path.join(dst_folder, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get all class names
    classes = os.listdir(src_folder)
    classes = [c for c in classes if os.path.isdir(os.path.join(src_folder, c))]

    # Split data and move to the corresponding folders
    for cls in classes:
        cls_dir = os.path.join(src_folder, cls)
        images = os.listdir(cls_dir)
        
        # Make sure there is more than two images to split
        if len(images) > 2:
            # Use train_test_split to partition the data
            train_images, test_images = train_test_split(images, test_size=3, random_state=42)
            val_images = test_images[:2]  # First image for validation
            test_images = test_images[1:]  # Second image for testing
        else:
            print(f"Not enough images for class {cls} to split. Needs more than 2 images.")
            continue
        
        # Function to copy files
        def copy_images(images, src_dir, dest_dir):
            cls_dest_dir = os.path.join(dest_dir, cls)
            os.makedirs(cls_dest_dir, exist_ok=True)
            for img in images:
                shutil.copy(os.path.join(src_dir, img), cls_dest_dir)
        
        # Copy images
        copy_images(train_images, cls_dir, train_dir)
        copy_images(val_images, cls_dir, val_dir)
        copy_images(test_images, cls_dir, test_dir)

# Define source and destination folders
src_folder = './faces'
dst_folder = './dataset_split'  # Destination folder is at the same level as 'faces'

# Run the function
split_data(src_folder, dst_folder)


---

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from facenet_pytorch import InceptionResnetV1

# Set the paths to the split dataset
train_dir = './dataset_split/train'
val_dir = './dataset_split/val'

# 1. Load and preprocess data
data_transforms = transforms.Compose([
    transforms.Resize((160, 160)),  # FaceNet's default input size is 160x160
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [3]:
import os
# 2. Create and modify the model
# Assuming you already know the number of classes
num_classes = len(os.listdir(train_dir))  # The number of folders in the training dataset is the number of classes
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=num_classes)

# 3. Define the training process
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 4. Train and validate the model
num_epochs = 5  # Specify the number of epochs
for epoch in range(num_epochs):
    model.train()
    for imgs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Simplified validation process
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, labels in val_loader:
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}, Accuracy: {100 * correct / total}%')

# Finally, save the model for later use
torch.save(model.state_dict(), './model_face_recognition.pth')

Epoch 1, Accuracy: 2.197802197802198%
Epoch 2, Accuracy: 2.197802197802198%
Epoch 3, Accuracy: 4.395604395604396%


KeyboardInterrupt: 

In [6]:
# Assuming you have saved your trained model to 'model_face_recognition.pth'
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=num_classes)
model.load_state_dict(torch.load('./model_face_recognition.pth'))
model.eval()

# You would need to have a 'test' folder similar to your 'train' and 'val' folders
test_dataset = datasets.ImageFolder(root='./dataset_split/test', transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        outputs = model(imgs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 5.617977528089888%


In [8]:
import cv2
from PIL import Image

# Map label indices to class names (assuming you have a mapping dictionary)
idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}

# Initialize the video feed
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    image = Image.fromarray(frame)
    image = data_transforms(image).unsqueeze(0)

    # Predictions
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        pred_class = idx_to_class[preds.item()]

    # Display the resulting frame with the prediction
    cv2.putText(frame, pred_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

: 